# 01. 퍼셉트론 (Perceptron)

가장 간단한 신경망인 퍼셉트론을 구현하고 이해합니다.

## 학습 목표
- 퍼셉트론의 구조와 작동 원리 이해
- 경사하강법(Gradient Descent) 이해
- 이진 분류 문제 해결

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# 시드 고정
torch.manual_seed(42)
np.random.seed(42)

## 1. 퍼셉트론이란?

퍼셉트론은 가장 간단한 형태의 인공 뉴런입니다.

$$y = \text{sign}(w \cdot x + b)$$

- $x$: 입력 벡터
- $w$: 가중치 벡터
- $b$: 편향(bias)
- $\text{sign}$: 부호 함수 (양수면 1, 음수면 -1)

## 2. 데이터 생성

선형 분리 가능한 2D 데이터를 생성합니다.

In [ ]:
# 데이터 생성
n_samples = 100

# 클래스 1: 중심 (2, 2)
class1 = torch.randn(n_samples // 2, 2) + torch.tensor([2.0, 2.0])
# 클래스 2: 중심 (-2, -2)
class2 = torch.randn(n_samples // 2, 2) + torch.tensor([-2.0, -2.0])

X = torch.cat([class1, class2], dim=0)
y = torch.cat([torch.ones(n_samples // 2), -torch.ones(n_samples // 2)])

# 데이터 시각화
plt.figure(figsize=(8, 6))
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Class 1', alpha=0.7)
plt.scatter(X[y == -1, 0], X[y == -1, 1], c='red', label='Class -1', alpha=0.7)
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.title('Binary Classification Data')
plt.grid(True, alpha=0.3)
plt.show()

## 3. 퍼셉트론 직접 구현

PyTorch의 autograd를 활용하지 않고, 퍼셉트론 학습 규칙을 직접 구현합니다.

In [ ]:
class PerceptronManual:
    """
    퍼셉트론 (수동 구현)
    
    퍼셉트론 학습 규칙:
    - 예측이 맞으면: 가중치 업데이트 없음
    - 예측이 틀리면: w = w + lr * y * x
    """
    
    def __init__(self, input_size: int, lr: float = 0.1):
        self.weight = torch.zeros(input_size)
        self.bias = torch.zeros(1)
        self.lr = lr
    
    def predict(self, x: torch.Tensor) -> torch.Tensor:
        """예측: sign(w·x + b)"""
        linear = x @ self.weight + self.bias
        return torch.sign(linear)
    
    def fit(self, X: torch.Tensor, y: torch.Tensor, epochs: int = 100):
        """학습"""
        n_samples = X.size(0)
        errors_per_epoch = []
        
        for epoch in range(epochs):
            errors = 0
            for i in range(n_samples):
                xi, yi = X[i], y[i]
                prediction = self.predict(xi)
                
                # 예측이 틀리면 업데이트
                if prediction != yi:
                    self.weight += self.lr * yi * xi
                    self.bias += self.lr * yi
                    errors += 1
            
            errors_per_epoch.append(errors)
            
            # 에러가 0이면 조기 종료
            if errors == 0:
                print(f"Converged at epoch {epoch + 1}")
                break
        
        return errors_per_epoch

In [ ]:
# 퍼셉트론 학습
perceptron = PerceptronManual(input_size=2, lr=0.1)
errors = perceptron.fit(X, y, epochs=100)

# 학습 결과 시각화
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(errors)
plt.xlabel('Epoch')
plt.ylabel('Number of Errors')
plt.title('Training Errors per Epoch')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# 결정 경계 시각화
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Class 1', alpha=0.7)
plt.scatter(X[y == -1, 0], X[y == -1, 1], c='red', label='Class -1', alpha=0.7)

# 결정 경계: w1*x1 + w2*x2 + b = 0
w = perceptron.weight
b = perceptron.bias
x1_range = torch.linspace(-5, 5, 100)
x2_boundary = -(w[0] * x1_range + b) / (w[1] + 1e-10)
plt.plot(x1_range, x2_boundary, 'g-', linewidth=2, label='Decision Boundary')

plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.title('Learned Decision Boundary')
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. mlfs 패키지 사용

이제 우리가 구현한 `mlfs` 패키지를 사용해봅니다.

In [ ]:
from mlfs.nn.models import Perceptron
from mlfs.nn.optim import SGD
from mlfs.nn.losses import BCEWithLogitsLoss

In [ ]:
# 레이블을 0, 1로 변환 (BCE 손실용)
y_binary = (y + 1) / 2  # -1, 1 -> 0, 1

# 모델 생성
model = Perceptron(input_size=2)
optimizer = SGD(model.parameters(), lr=0.1)
criterion = BCEWithLogitsLoss()

# 학습
losses = []
epochs = 100

for epoch in range(epochs):
    # 순전파
    logits = model(X)
    loss = criterion(logits, y_binary)
    
    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        predictions = model.predict(X)
        accuracy = (predictions == y_binary.long()).float().mean()
        print(f"Epoch {epoch + 1}: Loss = {loss.item():.4f}, Accuracy = {accuracy:.4f}")

In [ ]:
# 결과 시각화
from mlfs.utils.viz import plot_loss

plot_loss(losses, title='Training Loss (mlfs Perceptron)')

## 5. MNIST로 확장

실제 데이터셋인 MNIST에서 이진 분류를 수행해봅니다.

In [ ]:
from mlfs.utils.data import load_mnist

# MNIST 로드
X_train, y_train = load_mnist(train=True, flatten=True)
X_test, y_test = load_mnist(train=False, flatten=True)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

In [ ]:
# 이진 분류: 0 vs 1
def filter_binary(X, y, class0=0, class1=1):
    mask = (y == class0) | (y == class1)
    X_filtered = X[mask]
    y_filtered = y[mask]
    y_binary = (y_filtered == class1).float()
    return X_filtered, y_binary

X_train_bin, y_train_bin = filter_binary(X_train, y_train, 0, 1)
X_test_bin, y_test_bin = filter_binary(X_test, y_test, 0, 1)

print(f"Binary Train: {X_train_bin.shape}")
print(f"Binary Test: {X_test_bin.shape}")

In [ ]:
# 퍼셉트론 학습
model = Perceptron(input_size=784)
optimizer = SGD(model.parameters(), lr=0.01)
criterion = BCEWithLogitsLoss()

losses = []
train_accs = []
test_accs = []

epochs = 50
batch_size = 128

for epoch in range(epochs):
    # 미니배치 학습
    epoch_loss = 0
    n_batches = 0
    
    indices = torch.randperm(len(X_train_bin))
    
    for i in range(0, len(X_train_bin), batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = X_train_bin[batch_idx]
        y_batch = y_train_bin[batch_idx]
        
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        n_batches += 1
    
    avg_loss = epoch_loss / n_batches
    losses.append(avg_loss)
    
    # 정확도 계산
    with torch.no_grad():
        train_pred = model.predict(X_train_bin)
        train_acc = (train_pred == y_train_bin.long()).float().mean().item()
        
        test_pred = model.predict(X_test_bin)
        test_acc = (test_pred == y_test_bin.long()).float().mean().item()
    
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}: Loss = {avg_loss:.4f}, Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")

In [ ]:
# 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(losses)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(train_accs, label='Train')
axes[1].plot(test_accs, label='Test')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Test Accuracy: {test_accs[-1]:.4f}")

## 요약

이 노트북에서 배운 내용:

1. **퍼셉트론 구조**: 입력 → 가중치 합 → 활성화 함수 → 출력
2. **퍼셉트론 학습 규칙**: 틀린 예측에 대해서만 가중치 업데이트
3. **경사하강법**: 손실 함수를 최소화하는 방향으로 파라미터 업데이트
4. **한계**: 선형 분리 불가능한 문제(XOR)는 해결 불가

다음 노트북에서는 이 한계를 극복하는 **다층 퍼셉트론(MLP)**을 학습합니다.